# Настройка ноутбука

In [16]:
import pandas as pd
import numpy as np
from sklearn import datasets
import datetime as dt

from pandasql import sqldf

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Описание

## Ресурсы

[7 Advanced SQL Concepts You Need to Know!](https://medium.com/dp6-us-blog/7-advanced-sql-concepts-you-need-to-know-45fa149ba0b0)

# Концепты

## Порядок выполнения команд - execution order

Python - это императивный язык программирования, в нем мы описываем **как** сделать то, что нам нужно. А SQL это декларативный язык, в нем мы описываем **что** хотим получить. Разница между этими двумя подходами проявляется в последновательности исполнения комманд. В SQL они выполняются не в порядке их ввода (как в питоне), а в строго заданной последовательности:

[Ресурс](https://techrocks.ru/2021/03/05/order-of-sql-operations/)

1. FROM (выбор таблицы) / JOIN (комбинация с подходящими по условию данными из второй таблицы)
2. WHERE (фильтрация строк)
3. GROUP BY (агрегирование данных)
4. HAVING (фильтрация агрегированных данных)
5. SELECT (возврат результирующего датасета) / CASE (if-else выражения)
6. DISTINCT
7. UNION (объединение )
8. ORDER BY (сортировка)
9. LIMIT, TOP OFFSET

Как правило, запрос исполняется именно в такой последовательности.

Однако, в последних SQL-диалектах допускаются отходы от этой последовательности. Так в SQLite можно использовать в GROUP BY обозначения из SELECT. Хотя по идее это равносильно обращению к еще не заданной переменной. Это видно на примере ниже:

### Пример

In [4]:
df = pd.DataFrame({
    'name': ['Robert', 'Rojer', 'Rex', 'Alex', 'Jason', 'Thomas', 'Tobias'],
})
df.head()

,name
0,Robert
1,Rojer
2,Rex
3,Alex
4,Jason


In [5]:
# Почему это работает?
query = """
select
      substr(name, 1, 1) as first_letter
    , count(1) as n_rows
from
    df
group by 
    first_letter
order by
    n_rows desc
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,first_letter,n_rows
0,R,3
1,T,2
2,A,1
3,J,1


In [6]:
query = """
select
      substr(name, 1, 1) as first_letter
    , count(name) as n_rows
from
    df
group by 
    substr(name, 1, 1)
order by
    n_rows desc
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,first_letter,n_rows
0,R,3
1,T,2
2,A,1
3,J,1


## Подзапросы - subqueries

Есть три основных способа использования подзапросов:

### 1) Как источник данных для основного запроса
Эта практика обычно используется для получения подмножества или фильтрации записей.

#### Пример

In [7]:
df = pd.DataFrame({
    'name': ['Robert', 'Robert', 'Rex'],
    'surname': ['Pattison', 'Paulson', 'Roberts']
})

df.head()

,name,surname
0,Robert,Pattison
1,Robert,Paulson
2,Rex,Roberts


In [8]:
query = """
select
      surname
    , count(surname) as n_students
from
    (select surname from df where name = 'Robert') as t
group by 
    t.surname
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,surname,n_students
0,Pattison,1
1,Paulson,1


Технически мы неограничены в количестве уровней подзапросов. Однако читаемость кода уменьшается с каждым уровнем.

### 2) Внутри списка колонок выбранных в select-е

Это может быть полезно, когда для расчета нужных метрик необходимы данные из другой таблицы, при этом не хочется объединять таблицы

#### Пример

In [9]:
students = pd.DataFrame({
    'std_id': [1, 2, 3, 4, 5, 6],
    'name': ['Robert Pattison', 'Robert Paulson', 'Rex Roberts', 'Thomas Edison', 'Bonny Bones', 'Elison Woods'],
})

students.head()

,std_id,name
0,1,Robert Pattison
1,2,Robert Paulson
2,3,Rex Roberts
3,4,Thomas Edison
4,5,Bonny Bones


In [10]:
presence = pd.DataFrame({
    'std_id': [
        1, 2, 3, 4, 5, 6,
        1, 2, 3, 4, 5, 6,
        1, 2, 3, 4, 5, 6,
    ],
    'lesson_id': [
        1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2,
        3, 3, 3, 3, 3, 3,
    ],
    
    'is_present': [
        True, False, False, True, True, True,
        True, True, True, True, True, True,
        False, True, True, True, True, False,
    ],
})

presence.head()

,std_id,lesson_id,is_present
0,1,1,True
1,2,1,False
2,3,1,False
3,4,1,True
4,5,1,True


In [11]:
exams = pd.DataFrame({
    'std_id': [
        1, 2, 3, 4, 5, 6,
        1, 2, 3, 4, 5, 6,
    ],
    'exam_id': [
        1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2,
    ],
    'grade': [
        5, 2, 3, 4, 3, 5,
        4, 4, 4, 5, 4, 5,
    ],
})

exams.head()

,std_id,exam_id,grade
0,1,1,5
1,2,1,2
2,3,1,3
3,4,1,4
4,5,1,3


In [12]:
query = """
select
     name
    , (
        select avg(grade)
        from exams e
        where s.std_id = e.std_id
    
    ) as average_grades
    
    , (
        select count(is_present)
        from presence p
        where s.std_id = p.std_id and p.is_present = True
    ) as quantity_of_presence
from
    students s
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,name,average_grades,quantity_of_presence
0,Robert Pattison,4.5,2
1,Robert Paulson,3.0,2
2,Rex Roberts,3.5,2
3,Thomas Edison,4.5,3
4,Bonny Bones,3.5,3
5,Elison Woods,5.0,2


### 3) Как внешний фильтр запроса

#### Пример

In [13]:
students = pd.DataFrame({
    'std_id': [1, 2, 3, 4, 5, 6],
    'name': ['Robert Pattison', 'Robert Paulson', 'Rex Roberts', 'Thomas Edison', 'Bonny Bones', 'Elison Woods'],
})

students.head()

,std_id,name
0,1,Robert Pattison
1,2,Robert Paulson
2,3,Rex Roberts
3,4,Thomas Edison
4,5,Bonny Bones


In [14]:
presence = pd.DataFrame({
    'std_id': [
        1, 2, 3, 4, 5, 6,
        1, 2, 3, 4, 5, 6,
        1, 2, 3, 4, 5, 6,
    ],
    'lesson_id': [
        1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2,
        3, 3, 3, 3, 3, 3,
    ],
    
    'is_present': [
        True, False, False, True, True, True,
        True, True, True, True, True, True,
        False, True, True, True, True, False,
    ],
})

presence.head()

,std_id,lesson_id,is_present
0,1,1,True
1,2,1,False
2,3,1,False
3,4,1,True
4,5,1,True


In [15]:
query = """
select
      std_id
    , name
from
    students
where
    std_id in (select std_id from presence where is_present = True)
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,std_id,name
0,1,Robert Pattison
1,2,Robert Paulson
2,3,Rex Roberts
3,4,Thomas Edison
4,5,Bonny Bones
5,6,Elison Woods


В большинстве случаев можно добиться тех же результатов не прибегая к подзапросам, а используя join-ы. Однак, если основной запрос явно не использует колонки из другой таблицы, лучше воспользоваться постзапросами, поскольку процедура объединения довольно затратна.

#### Примеры задач

- [Stratascratch: Workers With The Highest Salaries - Medium](https://platform.stratascratch.com/coding/10353-workers-with-the-highest-salaries?code_type=1)

## Case

### Фильтрование значений перед агрегированием - Filtering values before aggregating

#### Примеры задач

- [Stratascratch: Salaries Differences - Easy](https://platform.stratascratch.com/coding/10308-salaries-differences?tabname=solutions)

### Введение колонки категориального признака - Introduction of a categorical attribute column

#### Примеры задач

- [Stratascratch: Classify Business Type - Medium](https://platform.stratascratch.com/coding/9726-classify-business-type?tabname=question)
- [Stratascratch: Host Popularity Rental Prices - Hard](https://platform.stratascratch.com/coding/9632-host-popularity-rental-prices?code_type=1)

## Табличные выражения - Common Table Expressions (CTEs)

Проблема подзапросов в том, что они вредят читаемости запросов. Для решения этой проблемы были введены табличные выражения. Они позволяют инкапсулировать отдельные части запроса в отдельные блоки и обращаться к ним из основного запроса. 

#### Прммер

In [127]:
df = pd.DataFrame({
    'name': ['Robert', 'Robert', 'Rex', 'Robert'],
    'surname': ['Pattison', 'Paulson', 'Roberts', 'De Niro']
})

df.head()

,name,surname
0,Robert,Pattison
1,Robert,Paulson
2,Rex,Roberts
3,Robert,De Niro


In [128]:
query = """
with robert_table as (
    select surname from df where name = 'Robert'
)

select
      surname
    , count(surname) as n_students
from
    robert_table
group by 
    robert_table.surname
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,surname,n_students
0,De Niro,1
1,Pattison,1
2,Paulson,1


In [129]:
query = """
with 

robert_table as (
    select surname from df where name = 'Robert'
),

surname_count as (
    select 
          surname
        , count(surname) as n_students 
    from 
        robert_table 
    group by 
        robert_table.surname
)


select
      surname
    , n_students
from 
    surname_count
where
    surname like 'P%'
      
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,surname,n_students
0,Pattison,1
1,Paulson,1


Особенность CTE в том, что после создания мы можем обащаться к каждому блоку из основного заброса. Что позволяет сильно его упростить.
Однако это работает только для первого запроса. Если выполнить сразу два запроса подряд, то второй выведет ошибку.

<img src="../../img/CTE.PNG" width="800">

## Оконные функции - window functions

В SQL есть несколько типов функций. Скалярные функции принимают на вход одно значение и возвращают тоже одно (например concat, upper, lower). Агрегирующие функции принимают на вход набор значений и возращают одно значение (sum, min, max, avg и т.д.). 

Есть также третий тип функций, которые принимают на вход набор строк, выполняют преобразования в контексте этого набора и возвращают приобразованные значения  - это оконные функции (RANK, DENSE_RANK, ROW_NUMBER). 

<img src="../../img/window_functions.PNG" width="800">

### Примеры задач

- [Stratascratch: Ranking Most Active Guests - Medium](https://platform.stratascratch.com/coding/10159-ranking-most-active-guests?tabname=question)
- [Stratascratch: Marketing Campaign Success [Advanced] - Hard](https://platform.stratascratch.com/coding/514-marketing-campaign-success-advanced?tabname=question)
- [Stratascratch: Monthly Percentage Difference - Hard](https://platform.stratascratch.com/coding/10319-monthly-percentage-difference?tabname=question)

### Пример

In [130]:
exams = pd.DataFrame({
    'name': [
        'Robert Pattison', 'Robert Paulson', 'Rex Roberts', 'Thomas Edison', 'Bonny Bones', 'Elison Woods',
        'Robert Pattison', 'Robert Paulson', 'Rex Roberts', 'Thomas Edison', 'Bonny Bones', 'Elison Woods',
    ],
    'exam_id': [
        1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2,
    ],
    'grade': [
        5, 2, 3, 4, 3, 5,
        4, 4, 4, 5, 4, 5,
    ],
})

exams.head()

,name,exam_id,grade
0,Robert Pattison,1,5
1,Robert Paulson,1,2
2,Rex Roberts,1,3
3,Thomas Edison,1,4
4,Bonny Bones,1,3


In [131]:
query = """
select 
      row_number() over () as row_number
    , name
    , grade
FROM 
    exams
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,row_number,name,grade
0,1,Robert Pattison,5
1,2,Robert Paulson,2
2,3,Rex Roberts,3
3,4,Thomas Edison,4
4,5,Bonny Bones,3
5,6,Elison Woods,5
6,7,Robert Pattison,4
7,8,Robert Paulson,4
8,9,Rex Roberts,4
9,10,Thomas Edison,5


In [132]:
query = """
select
      row_number() over (partition by name) as row_number
    , name
    , grade
FROM 
    exams
"""

pysqldf = lambda q: sqldf(q, globals())
pysqldf(query)

,row_number,name,grade
0,1,Bonny Bones,3
1,2,Bonny Bones,4
2,1,Elison Woods,5
3,2,Elison Woods,5
4,1,Rex Roberts,3
5,2,Rex Roberts,4
6,1,Robert Pattison,5
7,2,Robert Pattison,4
8,1,Robert Paulson,2
9,2,Robert Paulson,4


### Посчитать процентное изменение

```sql 
select 
      to_char(created_at::date, 'YYYY-MM') as year_month
    , round(
        (sum(value) - lag(sum(value),1) over (w)) *100/lag(sum(value),1) over (w), 2
    ) as revenue_diff
from df
group by year_month
window w as (order by to_char(created_at::date, 'YYYY-MM'))
```

### Отранжировать суммы

```sql 
select
      dense_rank() over (order by sum(value_col) desc)
    , id_col
    , sum(value_col) as values_sum
from df
group by id_col
```

## Скалярные пользовательские функции - User Defined Functions (UDF): Scalar functions

Как и в других языках программирования, в SQL вы также можете инкапсулировать логику в функции, которые принимают параметры, а возвращают значение или набор значений. Пользовательские функции используются в ситуациях, когда нашу задачу нельзя решить с помощью встроенных функций.

Как было сказано ранее, скалярные отличаются тем, что возвращают только одно число.

```sql
create function frequency_students (name_student string) returns int64 as (
    
    select 
        count(1)
    from 
        presence
    where
        name = name_student
        and
        is_present = True
    
);

create function average_students (name_student string) returns float64 as (

    select
        avg(grade)
    from
        exams
    where
        name = name_student

);

select
      std_id
    , name
    , average_students(name) as average_grades
    , frequency_students(name) as quantity_of_presence
from
    students
```

## Табличные пользовательские функции - User Defined Functions (UDF): Table functions

В отличии от скалярных функций, табличные функции принимают параметры и возвращают таблицу вместо одного значения. В отличае от CTE, табличные функции могут использоваться в контексте всей базы, а не только одного запроса.

```sql
create table function count_students_by_surname(name_student string, first_letter_surname string) as (

    with  

    student_filtered_by_name as (
        select surname from students where name = name_student
    ),

    count_of_students_by_surname as (
        select
              surname
            , count(1) as count_students
        from 
            students_filtered_by_name
        group by surname
    )

    select
          surnamae
        , count_students
    from
        count_of_students_by_name
    where
        surname like concat(first_letter_surname, '%')

);

select 
      surname
    , count_students
from
    count_students_by_surname('Robert', 'P')
```

## Временные таблицы - Temporary Tables

Как исходит из названия временные таблицы не хранятся в базе данных, а находятся в кеше и удаляются либо самим пользователем, либо когда закончится сессия.

В отличае от CTE и табличных функций, временные таблицы обладают характеристиками обычных таблиц - в них можно добавлять и удалять строки.

Основная функция временных таблиц - хранить и обрабатывать промежуточные результаты запроса, чтобы оптимизировать время выполнения/выделенные ресурсы более сложного запроса. Например, вместо применения вычислений к таблицам с миллионами строк для получения подмножества данных вы можете сначала отфильтровать это подмножество во временную таблицу, а затем выполнить вычисления. Таким образом, мы работаем только с теми данными, которые нам нужны, и не тратим ресурсы на обработку запросов.

Синтаксис создания временных таблиц похож с созданием обычных. Нужон только добавить temporary после create:

```sql
create temporary table name_of_table as (
    select [columns] from [table]
);
```